## Eigenfaces - Face Recognition

#### Importing necessary libraries

In [59]:
import os
import numpy as np
from PIL import Image
import cv2

#### Loading and transforming the images into vectors

In [60]:
def load_images(path):
    images = {}
    for file in filter(lambda file: not file.startswith('.'), os.listdir(path)):
        img = Image.open(os.path.join(path, file))
        idx = int(file.split('_')[0])
        if not idx in images:
            images[idx] = []
        images[idx].append(np.asarray(img))

    return images

def transform_to_vec(dataset):
    for person_face in dataset:
        dataset[person_face] = list(map(lambda img: img.reshape(-1,1), dataset[person_face]))
    return dataset

def compute_average(dataset):
    average_face = {}
    for person_face in dataset:
        person_faces = dataset[person_face]
        average_face[person_face] = np.sum(person_faces, axis=0) / len(person_faces)
    return average_face

def normalize_dataset(dataset, mean):
    for person_face in dataset:
        if type(mean) == dict:
            dataset[person_face] = list(map(lambda face: face - mean[person_face],dataset[person_face]))
        else:
            dataset[person_face] = list(map(lambda face: face - mean, dataset[person_face]))
    return dataset

def compute_covariance_matrix(dataset):
    cov_matrixes = {}
    for person_face in dataset:
        matrixes = list(map(lambda face: face * face.T, dataset[person_face]))
        for matrix in matrixes:
            if person_face not in cov_matrixes:
                cov_matrixes[person_face] = matrix
            else:
                cov_matrixes[person_face] += matrix
        cov_matrixes[person_face] *=  1 / len(dataset[person_face])
    return cov_matrixes

def compute_face_covariance_matrix(face):
    return face * face.T

def compute_face_eigen_matrix(face, num_eigs=14):
    cov_matrix = compute_face_covariance_matrix(face)
    values, vects = np.linalg.eig(cov_matrix)
    indices = []
    while len(indices) < num_eigs:
        index = np.argmax(values)
        np.delete(values,index)
        indices.append(index)
    # print(vects.shape)
    # print(vects[:,indices].shape)
    return  vects[:,indices]


def compute_eigen_matrixes(cov_matrixes, num_eigs=14):
    eigen_matrixes = {}
    for matrix in cov_matrixes:
        values, vects = np.linalg.eig(cov_matrixes[matrix])
        indices = [i for i,v in enumerate(values > 0.5) if v]
        indices = indices[:num_eigs]
        eigen_matrixes[matrix] =  vects[:,indices]
    return eigen_matrixes


def compute_eigenspace_distance(image, eigen_matrix, mean):
    identity = np.identity(eigen_matrix.shape[0])

    return (identity - eigen_matrix @ eigen_matrix.T) @ (image - mean)



def display_projection_as_face_image(projection):
    img = projection.real
    img = img.reshape( 40, 30)
    # print(img.astype(np.uint64))

    cv2.imwrite('test.png', np.uint8(img))
    

##### Computing the eigen vectors

In [61]:
DATA_PATH = '../data'
TRAIN_DATA_PATH = os.path.join(DATA_PATH, 'train')
TEST_DATA_PATH = os.path.join(DATA_PATH, 'test')

train_data = load_images(TRAIN_DATA_PATH)
train_data = transform_to_vec(train_data)

mean_train_faces = compute_average(train_data)
train_data = normalize_dataset(train_data, mean_train_faces)
cov_matrices = compute_covariance_matrix(train_data)
eigen_matrixes = compute_eigen_matrixes(cov_matrices)

##### Computing the test dataset performance


In [62]:
test_data = load_images(TEST_DATA_PATH)
test_data = transform_to_vec(test_data)
wrong = 0
correct = 0

for person_face_label in test_data:
    for face in test_data[person_face_label]:
        min_diff = None
        prediction = None
        for idx in eigen_matrixes:
            projected_face = compute_eigenspace_distance(face, eigen_matrixes[idx], mean_train_faces[idx])
            # display_projection_as_face_image(  mean_train_faces[matrix])
            # for seen_face in train_data[matrix]:
            #     projection = compute_projection(seen_face, eigen_matrixes[matrix],  mean_train_faces[matrix])
                # display_projection_as_face_image(projection)
            dist = np.linalg.norm(projected_face)
            if min_diff is None:
                min_diff = dist
                prediction = idx
            elif min_diff > dist:
                min_diff = dist
                prediction = idx
        if prediction == person_face_label:
            correct += 1
        else:
            wrong += 1
        print('Predicted: {} - Real: {}'.format(prediction, person_face_label))

print('Correct: {} - Wrong: {} - Accuracy: {:.2f}%'.format(correct, wrong, correct * 100 / (correct + wrong)))

Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 

#### Second approach of solving the problem

In [63]:
def compute_overall_covariance_matrix(dataset):
    cov_matrix =None
    count = 0

    for person_face in dataset:
        matrixes = list(map(lambda face: face * face.T, dataset[person_face]))
        for matrix in matrixes:
            if cov_matrix is None:
                cov_matrix = matrix
            else:
                cov_matrix += matrix
            count+=1
    cov_matrix *= 1 / count
    return cov_matrix

def compute_overall_mean_face(dataset):
    mean_face = None
    count = 1
    for person_face in dataset:
        faces =  dataset[person_face]
        for face in faces:
            if mean_face is None:
                mean_face = face.copy()
            else:
                mean_face += face
            count+=1
    mean_face = 1 / count * mean_face
    return mean_face

def compute_overall_eigenvectors(cov_matrix):
    values, vects = np.linalg.eig(cov_matrix)
    indices = [i for i,v in enumerate(values > 0.5) if v]
    eigenfaces =  vects[:,indices]
    return eigenfaces

        
def compute_projection(image, eigen_matrix, mean):
    # print('projection')
    # print(image.shape, eigen_matrix.shape)
    projection = mean
    for col in range(eigen_matrix.shape[1]):
        projection = projection + image * eigen_matrix[:,col].reshape(-1, 1)
    
    return projection

    # lambda_ = np.linalg.inv(eigen_matrix.T @ eigen_matrix) @ eigen_matrix.T @ image
    # return eigen_matrix @ lambda_

    # weights = []
    # for col in range(eigen_matrix.shape[1]):
    #     weights.append( eigen_matrix[:,col].reshape(-1, 1).T @ image)

    # return np.array(weights).reshape(-1,1)

In [64]:
train_data = load_images(TRAIN_DATA_PATH)
train_data = transform_to_vec(train_data)
mean_face = compute_overall_mean_face(train_data)
tain_data = normalize_dataset(train_data, mean_face)
overall_cov_matrix = compute_overall_covariance_matrix(train_data)
eigenfaces = compute_overall_eigenvectors(overall_cov_matrix)

test_data = load_images(TEST_DATA_PATH)
test_data = transform_to_vec(test_data)
test_data = normalize_dataset(test_data, mean_face)

wrong = 0
correct = 0

for person_face in test_data:
    for face in test_data[person_face]:
        min_diff = None
        prediction = None
        projected_face = compute_projection(face, eigenfaces, mean_face)
        for train_face in train_data:
            for seen_face in train_data[train_face]:
                projection = compute_projection(seen_face, eigenfaces, mean_face)
                diff = np.linalg.norm(projection - projected_face)
                if min_diff is None:
                    min_diff = diff
                    prediction = train_face
                elif min_diff > diff:
                    min_diff = diff
                    prediction = train_face
        if prediction == person_face:
            correct += 1
        else:
            wrong += 1
        print('Predicted: {} - Real: {}'.format(prediction, person_face))

print('Correct: {} - Wrong: {} - Accuracy: {:.2f}%'.format(correct, wrong, correct * 100 / (correct + wrong)))
            

Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 4 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 5 - Real: 5
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 4 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 